In [1]:
import ipyleaflet
from ipyleaflet import Map

# create map
basic_map = ipyleaflet.Map(zoom = 1)
# basic_map = Map(zoom=4)

# display map
basic_map
# display(basic_map)

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …